In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as cls
import seaborn as sns
import numpy as np
from pathlib import Path
from urllib.parse import urlparse
import glob
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.axis'] = 'y'  # Turns on gridlines by default
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['hatch.linewidth'] = 8
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
#plt.rcParams['axes.spines.bottom'] = False
#plt.rcParams['axes.spines.left'] = False
plt.rcParams.update({'text.color': "black",
                     'axes.labelcolor': "black"})
okabe_ito_palette = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#000000']
sns.set_palette(okabe_ito_palette)

In [3]:
pd.set_option('display.max_colwidth', None, 'display.max_rows', None)

forum = 'lesswrong'
topic_files = pd.read_csv(f'../src/metadata/clustering_results/{forum}/lda_55.csv')

topic_index = 20
n_rows = 20

topic_titles = topic_files[topic_files['dominant_topic'] == topic_index]['title']
sampled_topic_files = topic_titles.sample(n=n_rows)

print(sampled_topic_files)

11079                                                   What are examples of 'scientific' studies that contradict what you believe about yourself?
10490                                             We've built Connected Papers - a visual tool for researchers to find and explore academic papers
3253                                                                                              Is Caviar a Risk Factor For Being a Millionaire?
16650                                       Do universities have an incentive to ignore data ethics as more small sample bias is brought to light?
2284                                                                                               Link: Evidence-Based Medicine Has Been Hijacked
8910                                                                                                                 Acknowledgements & References
7762                                                                                             How often are new ide

In [4]:
"""Load all CSV files from the folder structure"""
print("Loading data from all CSV files...")

forum = 'lesswrong'
all_files = glob.glob(f"../src/processed_data/{forum}/02_with_links_and_gender/**/*.csv", recursive=True)
print(f"Found {len(all_files)} CSV files")

all_data = []
for file in all_files:
    try:
        df = pd.read_csv(file)
        # Extract year and month from filename
        parts = Path(file).stem.split('-')
        if len(parts) >= 2:
            df['year'] = int(parts[0])
            df['month'] = int(parts[1])
        all_data.append(df)
    except Exception as e:
        print(f"Error loading {file}: {e}")

if not all_data:
    print("No data found! Check your file paths.")

combined_df = pd.concat(all_data, ignore_index=True)
print(f"Loaded {len(combined_df)} total posts from {len(all_data)} files")

# Clean and prepare data
combined_df['postedAt'] = pd.to_datetime(combined_df['postedAt'], errors='coerce')
combined_df['baseScore'] = pd.to_numeric(combined_df['baseScore'], errors='coerce').fillna(0)
combined_df['commentCount'] = pd.to_numeric(combined_df['commentCount'], errors='coerce').fillna(0)

Loading data from all CSV files...
Found 198 CSV files
Loaded 45357 total posts from 198 files


In [ ]:
# nice plot templates
!wget https://raw.githubusercontent.com/h4pZ/rose-pine-matplotlib/main/themes/rose-pine-dawn.mplstyle -P /tmp

In [ ]:
def truncate_colormap(cmap, min_val=0.0, max_val=1.0, n=100):
    """
    Truncate the color map according to the min_val and max_val from the
    original color map.
    """
    new_cmap = cls.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=min_val, b=max_val),
        cmap(np.linspace(min_val, max_val, n)))
    return new_cmap

In [ ]:
combined_df["super_topic"] = combined_df["topic_label"].str.split(":", n=1).str[0].str.strip()
combined_df["sub_topic"] = combined_df["topic_label"].str.split(":", n=1).str[1].str.strip()

"""=== TOPIC ANALYSIS (Super vs Subtopics) ==="""

# --- Figure 1: Super topic distribution ---
super_counts = combined_df[combined_df["super_topic"] != 'Misc']["super_topic"].value_counts()

plt.figure(figsize=(12, 6))
plt.pie(
    super_counts.values,
    labels=super_counts.index,
    autopct='%1.1f%%',
    startangle=92,
    counterclock=False,
    colors = ["#8c2d04", "#cc4c02", "#ec7014", "#fe9929", "#fec44f", "#fee391", "#fff7bc"],
    wedgeprops = { 'linewidth' : 1, 'edgecolor' : 'white'}
)
plt.xticks(rotation=45, ha="right")
plt.title("Topic Distribution")
plt.tight_layout()
plt.savefig("img/topic_analysis_super.pdf", dpi=300, bbox_inches="tight")
plt.show()

base_color = "#f23e07"
cmap = LinearSegmentedColormap.from_list("grad_orange", ["#f0c472", base_color])

# Iterate over super_topics
for super_topic, group in combined_df[combined_df["super_topic"] != 'Misc'].groupby("super_topic"):
    sub_counts = group["sub_topic"].value_counts()

    fig, ax = plt.subplots()
    bars = ax.bar(sub_counts.index, sub_counts.values)

    y_min, y_max = ax.get_ylim()
    grad = np.atleast_2d(np.linspace(0, 1, 256)).T
    ax = bars[0].axes  # axis handle
    lim = ax.get_xlim()+ax.get_ylim()
    for bar in bars:
        bar.set_zorder(1)  # put the bars in front
        bar.set_facecolor("none")  # make the bars transparent
        x, _ = bar.get_xy()  # get the corners
        w, h = bar.get_width(), bar.get_height()  # get the width and height

        # Define a new color map.
        # For instance, if one bar only takes 10% of the y-axis, then the color
        # map will only use the first 10% of the color map.
        c_map = truncate_colormap(cmap, min_val=0, max_val=(h - y_min) / (y_max - y_min))

        # Let the imshow only use part of the color map
        ax.imshow(grad, extent=[x, x+w, h, y_min], aspect="auto", zorder=0,
                cmap=c_map)
        
    ax.axis(lim)
    ax.set_xticks(range(len(sub_counts)))
    ax.set_xticklabels(sub_counts.index, rotation=45, ha="right")
    ax.set_ylabel("Number of Posts")
    ax.set_title(f'Subtopics: "{super_topic}"')
    plt.savefig(f"img/topic_analysis_sub_{super_topic.replace(' ', '_')}.pdf",
                dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
"""Analyze gender distribution"""
print("\n=== GENDER ANALYSIS ===")

gender_counts = combined_df['user_gender'].value_counts()
print(f"\nGender Distribution:")
for gender, count in gender_counts.items():
    percentage = (count / len(combined_df)) * 100
    print(f"{gender.title()}: {count:,} ({percentage:.1f}%)")

# Gender posts pie chart
plt.pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%', startangle=90,
        colors=["#e76f51", "#2a9d8f", "#e9c46a"], 
        wedgeprops = { 'linewidth' : 1, 'edgecolor' : 'white'},
        counterclock=False
        )
plt.title('Gender Distribution of Posts')
plt.savefig("img/gender_dist_posts.pdf", dpi=300, bbox_inches="tight")
plt.show()

# Gender users pie chart
# Get unique users with their gender (remove duplicates)
unique_users = combined_df[['user.username', 'user_gender']].drop_duplicates()
user_gender_counts = unique_users['user_gender'].value_counts()

plt.pie(user_gender_counts.values, autopct='%1.1f%%', startangle=90,
        colors=["#e76f51", "#2a9d8f", "#e9c46a"], 
        labels=gender_counts.index,
        wedgeprops = { 'linewidth' : 1, 'edgecolor' : 'white'},
        counterclock=False
        )
plt.title('Gender Distribution of Users')
plt.savefig("img/gender_dist_users.pdf", dpi=300, bbox_inches="tight")
plt.show()

# Gender distribution by topics
gender_topic = combined_df[combined_df["super_topic"] != 'Misc'].groupby(['super_topic', 'user_gender']).size().unstack(fill_value=0)
gender_topic['total'] = gender_topic['gf'] + gender_topic['gm'] + gender_topic['-']
gender_topic['female_perc'] = (gender_topic['gf'] * 100) / gender_topic['total']
gender_topic['male_perc'] = (gender_topic['gm'] * 100) / gender_topic['total']
gender_topic['unknown_perc'] = (gender_topic['-'] * 100) / gender_topic['total']

gender_perc = gender_topic[['female_perc', 'male_perc', 'unknown_perc']]
gender_perc = gender_perc.rename(columns={'male_perc': 'male', 'female_perc': 'female', 'unknown_perc': 'unknown'})

ax = gender_perc.plot(kind="bar", figsize=(10,6), 
                       color=["#2a9d8f", "#e76f51", "#e9c46a"],
                       stacked=True,
                        )
plt.title('Gender Distribution by Topic')
plt.ylabel('Percentage of Posts')
plt.grid(axis='x')
plt.xlabel('Topic')
plt.legend(title='Gender', bbox_to_anchor=(1,1))
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("img/gender_posts.pdf", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
"""Analyze author activity patterns"""
print("\n=== AUTHOR ANALYSIS ===")

author_counts = combined_df['user.username'].value_counts()
print(f"\nTotal unique authors: {len(author_counts):,}")
print(f"Authors with only 1 post: {(author_counts == 1).sum():,} ({((author_counts == 1).sum() / len(author_counts) * 100):.1f}%)")
print(f"Authors with 10+ posts: {(author_counts >= 10).sum():,}")
print(f"Authors with 50+ posts: {(author_counts >= 50).sum():,}")

print(f"\nTop 10 Most Active Authors:")
for author, count in author_counts.head(10).items():
    print(f"{author}: {count} posts")

# Author activity distribution
post_ranges = ['1 post', '2-5 posts', '6-10 posts', '11-25 posts', '26-50 posts', '50+ posts']
counts = [
    (author_counts == 1).sum(),
    ((author_counts >= 2) & (author_counts <= 5)).sum(),
    ((author_counts >= 6) & (author_counts <= 10)).sum(),
    ((author_counts >= 11) & (author_counts <= 25)).sum(),
    ((author_counts >= 26) & (author_counts <= 50)).sum(),
    (author_counts >= 50).sum()
]

plt.bar(post_ranges, counts)
plt.title('Author Activity Distribution')
plt.ylabel('Number of Authors')
plt.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig("img/authors_per_number_posts.pdf", dpi=300, bbox_inches="tight")
plt.show()

# Posts per author histogram (log scale)
plt.hist(author_counts.values, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Posts per Author')
plt.ylabel('Number of Authors')
plt.title('Distribution of Posts per Author')
plt.yscale('log')
plt.tight_layout()
plt.savefig("img/posts_per_author_histogram.pdf", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
"""Analyze temporal trends"""
print("\n=== TEMPORAL TRENDS ===")

# Posts by year
yearly_posts = combined_df.groupby('year').size()
print(f"\nPosts by Year:")
for year, count in yearly_posts.items():
    print(f"{year}: {count:,} posts")

# Heatmap of posts by year and month
if 'month' in df.columns:
    heatmap_data = combined_df.groupby(['year', 'month']).size().unstack(fill_value=0)
    sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlOrRd')
    plt.title('Posts by Year and Month')
    plt.xlabel('Month')
    plt.ylabel('Year')

plt.tight_layout()
plt.savefig('img/activity_heatmap.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
"""Generate a comprehensive summary report"""
print("\n" + "="*60)
print("               LESSWRONG DATA SUMMARY REPORT")
print("="*60)

print(f"\n📊 OVERALL STATISTICS:")
print(f"   Total Posts: {len(combined_df):,}")
print(f"   Date Range: {combined_df['year'].min()}-{combined_df['year'].max()}")
print(f"   Unique Authors: {combined_df['user.username'].nunique():,}")
print(f"   Unique Topics: {combined_df['topic_label'].nunique()}")

print(f"\n👥 AUTHOR INSIGHTS:")
author_counts = combined_df['user.username'].value_counts()
print(f"   One-time contributors: {(author_counts == 1).sum():,} ({((author_counts == 1).sum() / len(author_counts) * 100):.1f}%)")
print(f"   Prolific authors (10+ posts): {(author_counts >= 10).sum():,}")
print(f"   Super authors (50+ posts): {(author_counts >= 50).sum():,}")
print(f"   Most prolific: {author_counts.index[0]} with {author_counts.iloc[0]} posts")

print(f"\n🏷️  TOPIC INSIGHTS:")
topic_counts = combined_df['topic_label'].value_counts()
print(f"   Most popular topic: {topic_counts.index[0]} ({topic_counts.iloc[0]:,} posts)")
print(f"   Topics with 100+ posts: {(topic_counts >= 100).sum()}")

print(f"\n🚻 GENDER DISTRIBUTION:")
gender_counts = combined_df['user_gender'].value_counts()
for gender, count in gender_counts.items():
    percentage = (count / len(combined_df)) * 100
    print(f"   {gender.title()}: {count:,} ({percentage:.1f}%)")

print(f"\n📈 ENGAGEMENT METRICS:")
print(f"   Average score: {combined_df['baseScore'].mean():.1f}")
print(f"   Average comments: {combined_df['commentCount'].mean():.1f}")
print(f"   Highest scoring post: {combined_df.loc[combined_df['baseScore'].idxmax(), 'title'][:50]}...")
print(f"   Most commented post: {combined_df.loc[combined_df['commentCount'].idxmax(), 'title'][:50]}...")

print(f"\n📅 TEMPORAL TRENDS:")
yearly_posts = combined_df.groupby('year').size()
peak_year = yearly_posts.idxmax()
print(f"   Peak posting year: {peak_year} with {yearly_posts[peak_year]:,} posts")
print(f"   Growth from {yearly_posts.index[0]} to {yearly_posts.index[-1]}: {((yearly_posts.iloc[-1] / yearly_posts.iloc[0] - 1) * 100):+.1f}%")

In [ ]:
print("Sample of extracted_links:")
print(combined_df['extracted_links'].head(10))
print(f"\nTotal rows with links: {combined_df['extracted_links'].notna().sum()}")

all_links = []
for links in combined_df['extracted_links'].dropna():
    if isinstance(links, str):
        # If links are stored as a string, you might need to parse them
        # Adjust this based on how your links are formatted
        if links.startswith('[') and links.endswith(']'):
            # If it's a string representation of a list
            try:
                import ast
                link_list = ast.literal_eval(links)
                all_links.extend(link_list)
            except:
                all_links.append(links)
        else:
            all_links.append(links)
    elif isinstance(links, list):
        all_links.extend(links)
    else:
        all_links.append(str(links))

print(f"Total number of links: {len(all_links)}")

# Clean and analyze links
clean_links = [link.strip() for link in all_links if link and str(link).strip() != '']

# 1. DOI Analysis
doi_pattern = r'(doi:|dx\.doi\.org|doi\.org)'
doi_links = [link for link in clean_links if re.search(doi_pattern, link, re.IGNORECASE)]
doi_count = len(doi_links)

print(f"\n=== DOI ANALYSIS ===")
print(f"Links containing DOI: {doi_count} ({doi_count/len(clean_links)*100:.1f}%)")

# 2. LessWrong Analysis  
lesswrong_links = [link for link in clean_links if 'lesswrong' in link.lower()]
lesswrong_count = len(lesswrong_links)

print(f"\n=== LESSWRONG ANALYSIS ===")
print(f"Links pointing to LessWrong: {lesswrong_count} ({lesswrong_count/len(clean_links)*100:.1f}%)")

# 3. Domain Analysis - Extract domains from all links
def extract_domain(url):
    try:
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
        parsed = urlparse(url)
        domain = parsed.netloc.lower()
        # Remove www. prefix for cleaner grouping
        if domain.startswith('www.'):
            domain = domain[4:]
        return domain
    except:
        return 'invalid_url'

domains = [extract_domain(link) for link in clean_links]
domain_counts = pd.Series(domains).value_counts()

print(f"\n=== TOP DOMAINS ===")
print("Top 15 domains:")
for domain, count in domain_counts.head(15).items():
    percentage = count/len(clean_links)*100
    print(f"{domain}: {count} links ({percentage:.1f}%)")

# 4. Visualization
plt.figure(figsize=(15, 10))

# Subplot 1: DOI vs Non-DOI
plt.subplot(2, 2, 1)
doi_data = ['DOI Links', 'Other Links']
doi_values = [doi_count, len(clean_links) - doi_count]
plt.pie(doi_values, labels=doi_data, autopct='%1.1f%%', startangle=90,
        colors=["#2a9d8f", "#e76f51"])
plt.title('Links with DOI vs Other Links')

# Subplot 2: LessWrong vs Others
plt.subplot(2, 2, 2)
lw_data = ['LessWrong', 'Other Sites']
lw_values = [lesswrong_count, len(clean_links) - lesswrong_count]
plt.pie(lw_values, labels=lw_data, autopct='%1.1f%%', startangle=90,
        colors=["#e9c46a", "#264653"])
plt.title('LessWrong Links vs Other Sites')

# Subplot 3: Top domains bar chart
plt.subplot(2, 2, 3)
top_domains = domain_counts.head(10)
plt.barh(range(len(top_domains)), top_domains.values)
plt.yticks(range(len(top_domains)), top_domains.index)
plt.xlabel('Number of Links')
plt.title('Top 10 Most Linked Domains')
plt.gca().invert_yaxis()

# Subplot 4: Link type categories
plt.subplot(2, 2, 4)
categories = {
    'DOI/Academic': doi_count,
    'LessWrong': lesswrong_count,
    'YouTube': len([d for d in domains if 'youtube' in d or 'youtu.be' in d]),
    'Wikipedia': len([d for d in domains if 'wikipedia' in d]),
    'Arxiv': len([d for d in domains if 'arxiv' in d]),
    'EA Forum': len([d for d in domains if 'forum.effectivealtruism' in d]),
    'Alignmentforum': len([d for d in domains if 'alignmentforum' in d]),
    'Other': len(clean_links) - doi_count - lesswrong_count - 
             len([d for d in domains if 'youtube' in d or 'youtu.be' in d]) -
             len([d for d in domains if 'wikipedia' in d])
}

plt.pie(categories.values(), labels=categories.keys(), autopct='%1.1f%%', startangle=90)
plt.title('Link Categories')

plt.tight_layout()
plt.savefig("img/link_analysis.pdf", dpi=300, bbox_inches="tight")
plt.show()

# 5. Additional insights
print(f"\n=== ADDITIONAL INSIGHTS ===")
youtube_count = len([d for d in domains if 'youtube' in d or 'youtu.be' in d])
wikipedia_count = len([d for d in domains if 'wikipedia' in d])
github_count = len([d for d in domains if 'github' in d])

print(f"YouTube links: {youtube_count} ({youtube_count/len(clean_links)*100:.1f}%)")
print(f"Wikipedia links: {wikipedia_count} ({wikipedia_count/len(clean_links)*100:.1f}%)")
print(f"GitHub links: {github_count} ({github_count/len(clean_links)*100:.1f}%)")

# Show some example links for verification
print(f"\n=== SAMPLE LINKS FOR VERIFICATION ===")
if doi_links:
    print(f"Sample DOI links: {doi_links[:3]}")
if lesswrong_links:
    print(f"Sample LessWrong links: {lesswrong_links[:3]}")

In [5]:
unknown_gender = combined_df[combined_df["user_gender"] == '-']
unknown_gender_counts = unknown_gender['user.username'].value_counts()
# if (unknown_gender_counts < 5).any():
unknwons = set(unknown_gender_counts.index)
unknowns_lower = {name.lower() for name in unknwons}

In [6]:
for user in unknowns_lower:
    print(user)

rcu
denreik
kohaku-none
funkycoder
lexico
rangi
jooyous
solipsist
fiddlemath
carn
zhaowei zhang
kytael
harfe
acrackedpot
elec0
expipiplusone
ar-sht
jbay
onewhofrogs
htlou
minibearrex
alvaromoret
eniscien
roxolan
cfar team
yanni
drocta
racinkc1
ahbwramc
dal
ekkolapto
gcmac
verbalshadow
furcas
9eb1
flowerfeatherfocus
spookyuser
bramflakes
edanm
en kepeig
shiroe
ratswrongaboutuap
realitygrill
avery-liu
demeter
xfrequentist
calecute
smitop
ete
irenictruth
schrodingart
tmfow
disoha
kei
kiboneu
rorschak
eugman
qbee
senarin
thetwelve
af2022
physicaleconomics
lionhearted
tanagrabeast
dentosal
caffemacchiavelli
tsang chung shu
ratcourse
lejuletre
randomini
wheelspawn
summerstay
aspiringknitter
ajfox
creatorrr
ahartntkn
askwho
jmiller
jpyykko
natural_number
hu-yichao
ndee
caesium
mnoetel
borasko
dripgrind
d𝜋
zlrth
milkandcigarettes
kingreaper
petruchio
mushroomsoup
hairyfigment
person-1
erfeyah
emr
sark
perplexed
purge
xpostah
elo
hangedman
wiserd
bayareahuman
twistingfingers
networked
nanashi
r

In [ ]:
import json
with open("../src/metadata/graphql_usernames.json", "r", encoding="utf-8") as f:
        names_data = json.load(f)

MALE_USERNAMES = names_data["MALE_USERNAMES"]
FEMALE_USERNAMES = names_data["FEMALE_USERNAMES"]

KeyError: 'MALE_NAMES'

In [ ]:
unknowns_without_male = unknowns_lower.difference(names.MALE_NAMES)
unknowns_without_female = unknowns_lower.difference(names.FEMALE_NAMES)
print(f'Unkown names that are neither in FEMALE_NAMES nor MALE_NAMES: {len(unknowns_without_male.intersection(unknowns_without_female))}')
print(unknowns_without_male.intersection(unknowns_without_female))

In [ ]:
male_names_without_unknowns = names.MALE_NAMES.difference(unknowns_lower)
print(sorted(male_names_without_unknowns))
print(sorted(names.MALE_NAMES - male_names_without_unknowns))
print(sorted(names.MALE_NAMES - names.MALE_USERNAMES))

In [ ]:
import importlib
import src.names as names
importlib.reload(names) 

In [ ]:
female_names_without_unknowns = names.FEMALE_NAMES.difference(unknowns_lower)
print(female_names_without_unknowns)
print(names.FEMALE_NAMES - female_names_without_unknowns)
print(names.FEMALE_NAMES - names.FEMALE_USERNAMES)

In [ ]:

import src.names as names
 # Force reload
print(type(names.MALE_NAMES))  # Should be <class 'set'>
print(len(names.MALE_NAMES))   # Check the size
print('yashvardhan' in names.MALE_NAMES)

In [ ]:
print(sorted(names.MALE_NAMES))

In [ ]:
purely_unknown = unknwons - names.MALE_USERNAMES

In [ ]:
purely_unknown_lower = {name.lower() for name in purely_unknown}
male_names_lower = {name.lower() for name in names.MALE_USERNAMES}
print(purely_unknown_lower.intersection(male_names_lower))

In [ ]:
all_names = list(names.FEMALE_NAMES) + list(names.MALE_NAMES)
all_names_sorted = sorted(all_names, key=len, reverse=True)

gf = []
gf_user = []
gm = []
gm_user = []

for username in unknowns_lower:
    for name in all_names_sorted:
        if len(name) > 3 and name in username:
            if name in names.FEMALE_NAMES:
                print(f"FEMALE: {username} with {name}")
                gf_user.append(username)
                gf.append(name)
            elif name in names.MALE_NAMES:
                print(f"MALE: {username} with {name}")
                gm_user.append(username)
                gm.append(name)

In [ ]:
print(gf_user)
print(len(gf_user))
print(gm_user)
print(len(gm_user))

In [ ]:
current_unknowns = unknowns_lower - set(gf_user) - set(gm_user)
for name in current_unknowns:
    print(name)

In [ ]:
perplexity = [1150.00, 1105.33, 1072.61, 1054.01, 1040.12, 1028.32, 1018.51, 1009.65, 1002.37]
topics = [10, 15, 20, 25, 30, 35, 40, 45, 50]
log_likelihood = [-126311522.90,-125601505.71, -125062974.37, -124749334.57, -124511617.32,  -124307203.07, -124135376.41, -123978846.41, -123849036.87]
plt.plot(topics, perplexity)

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('exp', color=color)
ax1.plot(topics, perplexity, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second Axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('sin', color=color)  # we already handled the x-label with ax1
ax2.plot(topics, log_likelihood, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()